In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import twitter_samples

In [3]:
df_ca = pd.read_csv('../Data/Cleaned/paid_ca.csv')

df_chile = pd.read_csv('../Data/Cleaned/paid_chile.csv')

df_nepal = pd.read_csv('../Data/Cleaned/paid_nepal.csv')

df_pak = pd.read_csv('../Data/Cleaned/paid_pak.csv')

In [3]:
df = pd.concat([df_ca, df_chile, df_nepal, df_pak])
df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)

In [4]:
df.shape

(8517, 3)

In [5]:
def binarize_label(df):
    df['label_is_relevent'] = df['label'].apply(lambda x: 0 if x == 'not_related_or_irrelevant' else 1)
    return df

df = binarize_label(df)

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
df.head(1)

,tweet_id,tweet_text,label,label_is_relevent
0,'504075795435094016',bridges overpasses earn approval after norther...,infrastructure_and_utilities_damage,1


In [7]:
df = df.drop(columns=['label', "tweet_id"])

In [8]:
tvec = TfidfVectorizer(max_features=25000, ngram_range=(1,2), stop_words="english")

In [9]:
X = df.tweet_text
y = df.label_is_relevent

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

tvec.fit(X_train)

logr = LogisticRegression()
logr.fit(tvec.transform(X_train).todense(), y_train)

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
cross_val_score(logr, tvec.transform(X).todense(), y).mean()

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8913936832217916

In [11]:
y.mean()

0.8901021486438887

In [12]:
cross_val_score(logr, tvec.transform(X_test).todense(), y_test).mean()

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8901408450704226

In [13]:
y_test.mean()

0.8901408450704226

In [14]:
tw_txt_list = []
strings = twitter_samples.strings('tweets.20150430-223406.json')
for string in strings[:9_000]:
    tw_txt_list.append(string)

tw_tw_id = ['007'] * 9_000
tw_label = ['dont care']* 9_000
tw_lbl_rel = [0]* 9_000

samples = {'tweet_id':tw_tw_id, 'tweet_text':tw_txt_list, 'label': tw_label, 'label_is_relevent':tw_lbl_rel}
twitter_samples = pd.DataFrame(samples)

In [15]:
twitter_samples.head()

,tweet_id,tweet_text,label,label_is_relevent
0,007,RT @KirkKus: Indirect cost of the UK being in ...,dont care,0
1,007,VIDEO: Sturgeon on post-election deals http://...,dont care,0
2,007,RT @LabourEoin: The economy was growing 3 time...,dont care,0
3,007,RT @GregLauder: the UKIP east lothian candidat...,dont care,0
4,007,RT @thesundaypeople: UKIP's housing spokesman ...,dont care,0


In [16]:
twitter_samples.shape

(9000, 4)

In [17]:
y.value_counts()

1    7581
0     936
Name: label_is_relevent, dtype: int64

In [18]:
7581-936

6645

In [19]:
df = pd.concat([df,twitter_samples.iloc[:6645, :]])

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [20]:
df.head(1)

,label,label_is_relevent,tweet_id,tweet_text
0,NaN,1,'504075795435094016',bridges overpasses earn approval after norther...


In [21]:
df['label_is_relevent'].mean()

df.reset_index(inplace=True)
df.drop(columns='index', inplace=True)

df = cleaning(df, 'tweet_text')

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [22]:
X = df.tweet_text
y = df.label_is_relevent

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

tvec.fit(X_train)

logr = LogisticRegression()
logr.fit(tvec.transform(X_train).todense(), y_train)

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [74]:
# df["label"] = df["label_is_relevent"]
# df.drop(columns="tweet_id", inplace = True)
df.to_csv("Train & Test/eq_tweet_train.csv", index=False)

In [23]:
cross_val_score(logr, tvec.transform(X).todense(), y, cv=5).mean()

C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Owner\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence t

0.8962076240601242

In [64]:
import keras

import numpy as np

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense

In [25]:
classifier = Sequential()

In [26]:
classifier.add(Dense(units = 50,
                     kernel_initializer = 'uniform', 
                     activation = 'relu',
                     input_dim=25000)
              )

classifier.add(Dense(units = 50,
                     kernel_initializer = 'uniform', 
                     activation = 'relu'
                    )
              )

classifier.add(Dense(units = 1,
                     kernel_initializer = 'uniform', 
                     activation = 'sigmoid'
                    )
              )

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [27]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
classifier.fit(tvec.transform(X).todense(), y, batch_size = 10, epochs = 5)

Epoch 1/5
15162/15162 [==============================] - 41s 3ms/sample - loss: 0.1697 - acc: 0.9357
Epoch 2/5
15162/15162 [==============================] - 34s 2ms/sample - loss: 0.0533 - acc: 0.98111s - loss
Epoch 3/5
15162/15162 [==============================] - 34s 2ms/sample - loss: 0.0213 - acc: 0.9930
Epoch 4/5
15162/15162 [==============================] - 34s 2ms/sample - loss: 0.0104 - acc: 0.9961
Epoch 5/5
15162/15162 [==============================] - 34s 2ms/sample - loss: 0.0059 - acc: 0.9977


In [29]:
testing = pd.read_csv("../earthquake.csv")
testing.drop(columns="Unnamed: 0", inplace=True)

In [30]:

testing['tweet_text'] = testing['text']
testing['relevant'] = testing['Unnamed: 2']

testing_clean = testing.drop(columns=['text', 'Unnamed: 2'])
testing_clean.head(1)

,tweet_text,relevant
0,Ml 2.54 earthquake (reviewed) occured at 2019-...,1


In [31]:
testing_clean = cleaning(testing_clean, 'tweet_text')

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [32]:
testing_clean.relevant.value_counts()

1    590
0     65
Name: relevant, dtype: int64

In [33]:
590-65

525

In [34]:
from nltk.corpus import twitter_samples
tw_txt_list = []
strings = twitter_samples.strings('tweets.20150430-223406.json')
for string in strings[:525]:
    tw_txt_list.append(string)

tw_tw_id = ['007'] * 525
tw_label = ['dont care']* 525
tw_lbl_rel = [0]* 525

new_samples = { 'tweet_text':tw_txt_list, 'relevant':tw_lbl_rel}
new_twitter_samples = pd.DataFrame(new_samples)

In [35]:
new_twitter_samples.head(1)

,tweet_text,relevant
0,RT @KirkKus: Indirect cost of the UK being in ...,0


In [36]:
testing_clean.head(1)

,tweet_text,relevant
0,ml 254 earthquake (reviewed) occured at 2019-1...,1


In [37]:
clean_test = pd.concat([testing_clean, new_twitter_samples])
clean_test.reset_index(inplace=True)
clean_test.drop(columns='index', inplace=True)

clean_test = cleaning(clean_test, "tweet_text")

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [38]:
clean_test.relevant.mean()

0.5

In [73]:
clean_test.to_csv("Train & Test/eq_tweets_test.csv", index=False)

In [39]:
preds = classifier.predict_on_batch(np.array(tvec.transform(clean_test.tweet_text).todense()))

In [40]:
from sklearn.metrics import accuracy_score

In [41]:
y = clean_test.relevant

In [42]:
preds = preds > 0.5

In [43]:
accuracy_score(y, preds)

0.9720338983050848

In [72]:
locations = pd.read_csv("../tweet_located.csv")

In [45]:
locations = cleaning(locations, "tweet_text")

C:\Users\Owner\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [46]:
locations.head(1)

,tweet_text,Time,origin_long,origin_lat,latitude,longitude
0,ml 31 earthquake (automatic) occured at 2019-1...,2019-10-31T18:30:57.638Z,-165.6256,65.4374,65.4374,-165.6256


In [47]:
preds = classifier.predict_on_batch(np.array(tvec.transform(locations.tweet_text).todense()))

In [48]:
locations["predicted_relevant"] = (preds > 0.5).astype("int64")

In [49]:
locations.to_csv("../test_file.csv",index=False)

In [50]:
import pickle

In [65]:
classifier.save("cnn_model")

In [66]:
new_model = Sequential()

new_model = load_model("cnn_model.h5")

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
